In [1]:
import numpy
import pandas
import matplotlib.pyplot as pyplot
from matplotlib.font_manager import FontProperties
import seaborn

import nltk
from nltk.corpus import stopwords,wordnet
from nltk.probability import FreqDist

from wordcloud import WordCloud
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI

import pickle

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# %matplotlib qt
%matplotlib inline
pyplot.rcParams["figure.figsize"] = (15, 12)

In [4]:
filename="Data.csv"

In [21]:
data=pandas.read_csv('../Data/Object_Data/'+filename)

In [22]:
data

,Questions,Answers,keywards,MainQuestion
0,कठिन सवाल,माफ करना मुझे इस सवाल का जवाब देने पे कठिनाई ...,मुश्कील,कठिन सवाल
1,महाविद्यालय कॉलेज का पता अड्रेस क्या हैं colle...,डॉ. के.एम. वासुदेवन पिल्लै परिसर प्लॉट नंबर 10...,अड्रेस ऑफिस,महाविद्यालय का पता अड्रेस क्या हैं?
2,महाविद्यालय कॉलेज का फोन टेलीफोन नंबर क्या हैं...,02227456030,दूरभाष दूरध्वनी,महाविद्यालय का टेलीफोन नंबर क्या हैं?
3,महाविद्यालय कॉलेज कब स्थापित स्थापना हुआ था co...,१९९९,स्थापना,महाविद्यालय कब स्थापित स्थापना हुआ था ?
4,कॉलेज कितना बड़ा है college dikhne me kaisa ha...,कॉलेज प्राकृतिक सुंदरता से घिरी 10 एकड़ भूमि म...,NaN,कॉलेज कितना बड़ा है
5,इंजीनियरिगं में कितने विभाग ब्रैंचेस हैं engin...,6 विभाग हैं,NaN,इंजीनियरिगं में कितने विभाग ब्रैंचेस हैं
6,दिन मे कॉलेज कितने बजे कब तक होता है din me co...,शाम ४ या ५ या ६ बजे तक राहत हैं,NaN,दिन मे कॉलेज कितने बजे तक होता है?
7,लेक्चर व्याख्यान कितने घंटे चलता है lectures k...,१ घंटा लगभग,NaN,व्याख्यान कितने घंटे चलता है?
8,प्रैक्टिकल लैब कितने घंटे चलता है practical pr...,२ घंटा लगभग,NaN,प्रैक्टिकल कितने घंटे चलता है?
9,कॉलेज कैंपस में वाईफाई हैं क्या wifi hain kya ...,हा |हमारे पास एक मुफ्त और सुरक्षित वाईफाई कनेक...,NaN,कैंपस में वाईफाई हैं या?


## Input Preprocessing Functions

In [23]:
NumberofCombinations=1

In [24]:
Regular_expression_definition_for_digits=re.compile('\d+\s|\s\d+|\s\d+\s')
Defined_Stopwords=["मैं","मुझको","मेरा","अपने आप को","हमने","हमारा","अपना","हम","आप","आपका","तुम्हारा","अपने आप","स्वयं","वह","इसे","उसके","खुद को","कि वह","उसकी","उसका","खुद ही","यह","इसके","उन्होने","अपने","क्या","जो","किसे","किसको","कि","ये","हूँ","होता है","रहे","थी","थे","होना","गया","किया जा रहा है","किया है","है","पडा","होने","करना","करता है","किया","रही","एक","लेकिन","अगर","या","क्यूंकि","जैसा","जब तक","जबकि","की","पर","द्वारा","के लिए","साथ","के बारे में","खिलाफ","बीच","में","के माध्यम से","दौरान","से पहले","के बाद","ऊपर","नीचे","को","से","तक","से नीचे","करने में","निकल","बंद","से अधिक","तहत","दुबारा","आगे","फिर","एक बार","यहाँ","वहाँ","कब","कहाँ","क्यों","कैसे","सारे","किसी","दोनो","प्रत्येक","ज्यादा","अधिकांश","अन्य","में कुछ","ऐसा","में कोई","मात्र","खुद","समान","इसलिए","बहुत","सकता","जायेंगे","जरा","चाहिए","अभी","और","कर दिया","रखें","का","हैं","इस","होता","करने","ने","बनी","तो","ही","हो","इसका","था","हुआ","वाले","बाद","लिए","सकते","इसमें","दो","वे","करते","कहा","वर्ग","कई","करें","होती","अपनी","उनके","यदि","हुई","जा","कहते","जब","होते","कोई","हुए","व","जैसे","सभी","करता","उनकी","तरह","उस","आदि","इसकी","उनका","इसी","पे","तथा","भी","परंतु","इन","कम","दूर","पूरे","गये","तुम","मै","यहां","हुये","कभी","अथवा","गयी","प्रति","जाता","इन्हें","गई","अब","जिसमें","लिया","बड़ा","जाती","तब","उसे","जाते","लेकर","बड़े","दूसरे","जाने","बाहर","स्थान","उन्हें ","गए","ऐसे","जिससे","समय","दोनों","किए","रहती","इनके","इनका","इनकी","सकती","आज","कल","जिन्हें","जिन्हों","तिन्हें","तिन्हों","किन्हों","किन्हें","इत्यादि","इन्हों","उन्हों","बिलकुल","निहायत","इन्हीं","उन्हीं","जितना","दूसरा","कितना","साबुत","वग़ैरह","कौनसा","लिये","दिया","जिसे","तिसे","काफ़ी","पहले","बाला","मानो","अंदर","भीतर","पूरा","सारा","उनको","वहीं","जहाँ","जीधर","के","एवं","कुछ","कुल","रहा","जिस","जिन","तिस","तिन","कौन","किस","संग","यही","बही","उसी","मगर","कर","मे","एस","उन","सो","अत"]
Hindi_Stop_Words=tuple(set(STOP_WORDS_HI).union(set(Defined_Stopwords)))
def preprocessing_of_sentence(text):
    text=Regular_expression_definition_for_digits.sub(r" ",text)
    punctuations = [".",",","!","?","'",'"',":",";","*","-","/","+","%","$","#","@","(",")","[","]","{","}"]
    for i in punctuations:
        text = text.replace(i," ")
    text=text.lower().split()
#     text=[word for word in text if word not in Hindi_Stop_Words and len(word)>1]
    return text

## Output Processing Functions

In [25]:
def getOutput_fromCosineSimilarity(array):
    array_element_sums=[]
    for i in array:
        array_element_sums.append(sum(i))
    return array_element_sums.index(max(array_element_sums))

# Vectorization

### 1. Normal Vectorization

In [26]:
Vectorizer=TfidfVectorizer()

In [27]:
Total_Vector_Data=Vectorizer.fit_transform(data['Questions'])

In [28]:
Total_Vector_Data

<14x94 sparse matrix of type '<class 'numpy.float64'>'
	with 182 stored elements in Compressed Sparse Row format>

In [29]:
Total_Vector_Data=Total_Vector_Data.toarray()

In [30]:
Total_Vector_Data

array([[0.        , 0.        , 0.        , ..., 0.        , 0.70710678,
        0.        ],
       [0.        , 0.        , 0.62741324, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.29709738, 0.        , ..., 0.29709738, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.24371811]])

In [31]:
with open("../Models/Vectorizer.pickle","wb") as f:
    pickle.dump(Vectorizer,f)

## Input Preparation

In [32]:
input_Text="दिन मे कॉलेज कब तक होता हैं ?"
input_Text=preprocessing_of_sentence(input_Text)
print(input_Text)
input_Text=Vectorizer.transform(input_Text)
print(input_Text.toarray())

['दिन', 'मे', 'कॉलेज', 'कब', 'तक', 'होता', 'हैं']
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0

# Model Implementation

## 1. Cosine Similarity

In [33]:
cosine_similarity_values=cosine_similarity(Total_Vector_Data,input_Text)

In [34]:
cosine_similarity_values

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.23493336, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.22504935, 0.26001844,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0. 

In [35]:
getOutput_fromCosineSimilarity(cosine_similarity_values)

6

In [20]:
print(data['Answers'][getOutput_fromCosineSimilarity(cosine_similarity_values)])

 शाम ४ या ५ या ६ बजे तक राहत हैं
